In [16]:
import sys
import matplotlib.pyplot as plt

from contextualbandits import online as cb
from copy import deepcopy as clone
import numpy as np
import pandas as pd
# The only classifiers that have partial_fit
# See: https://scikit-learn.org/0.15/modules/scaling_strategies.html
from sklearn.linear_model import SGDClassifier, PassiveAggressiveClassifier, Perceptron
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
sys.path.append("../src")

%matplotlib inline

In [5]:
def keep_values_by_min_freq(df, col, freq=0.9,other="OTHER"):
    ds = df[col].value_counts(normalize=True).cumsum()<freq
    values = set(ds[ds].index)
    df.loc[~(df[col].isin(values)), col]=other
    return df
def preprocess(df):
    df = df.drop("sessionId", axis=1)
    df["ts"] = pd.to_datetime(df["ts"]*1000000)
    df["day_of_week"]=df["ts"].dt.weekday.astype("category")
    df["time_of_day"]=df["ts"].dt.hour.astype("category")
    df = df.drop("ts", axis=1)
    df = keep_values_by_min_freq(df, "networkId")
    a = df["bidPrice"].round(2)
    r = df["hasWon"]
    df = df.drop(["bidPrice", "hasWon"],axis=1)    
    X = pd.get_dummies(df)
    return X,a, r
df = pd.read_csv("../data/data.csv")
# sample
df = df.query("placementType=='banner' and bidPrice<1.01").drop("placementType",axis=1)
X, a, r = preprocess(df)
X

,networkId_59,networkId_108,networkId_115,networkId_172,networkId_194,networkId_235,networkId_254,networkId_278,networkId_307,networkId_322,...,time_of_day_14,time_of_day_15,time_of_day_16,time_of_day_17,time_of_day_18,time_of_day_19,time_of_day_20,time_of_day_21,time_of_day_22,time_of_day_23
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2607670,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2607671,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2607672,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2607673,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


100

In [12]:
bids = sorted(set(a))
n_arms = len(bids)
logreg_ts = cb.BootstrappedTS(SGDClassifier(loss='log'), n_arms, batch_train=True)
#logreg_ts.partial_fit(X,a,r)

In [24]:

def simulate(model, X, a , r, chunk = 100):
    train_X, train_a, train_r    = X.iloc[:chunk,:], a[:chunk], r[:chunk]
    X, a, r = X.iloc[chunk:,:], a[chunk:], r[chunk:]
    test_X, test_a, test_r = X.iloc[:chunk,:], a[:chunk], r[:chunk]
    while len(X)>chunk:
        model.partial_fit(train_X,train_a,train_r)
        print (model.predict(test_X))
        1/0
        train_X, train_a, train_r    = X.iloc[:chunk,:], a[:chunk], r[:chunk]
        X, a, r = X.iloc[chunk:,:], a[chunk:], r[chunk:]
        test_X, test_a, test_r = X.iloc[:chunk,:], a[:chunk], r[:chunk]
    pass

In [ ]:
logreg_ts.

In [25]:
simulate(logreg_ts, X,a,r)

[51 46 42 67 61 37  5 75 25 47 18 63  6 42 29 79 17 24 50 95 70 11 38 22
 32 65 62 12  0 67  2 84 91 50 77 54 81 19 35 69 94 59 12 24 57 86 38 49
 90 74 19 70 93 86 63 37 23 98 52 99 41 90 47 75 62 75 51 90 87 27  3 30
 98 15 76 59 58 24 39 86 17  3 84 12 13 19 48 27 80 25 14 93 10 62 15 50
 36 62 28 16]


ZeroDivisionError: division by zero

In [21]:
head_a

0      0.44
2      0.05
3      0.39
4      0.17
5      0.89
       ... 
117    0.31
118    0.11
119    0.05
120    0.18
121    0.36
Name: bidPrice, Length: 100, dtype: float64